# AWS Lambda Operations - Comprehensive Guide

This notebook provides a complete walkthrough of AWS Lambda serverless compute operations using boto3 SDK.

## Prerequisites - Manual AWS Setup

Before running this notebook, the following setup was completed manually in AWS Console:

### Step 1: Create IAM Role for Lambda Execution
- Logged into AWS account
- Navigate to **IAM** > **Roles** > **Create role**
- Select trusted entity: **AWS service** > **Lambda**
- Attach policies:
  - **AWSLambdaBasicExecutionRole** (CloudWatch Logs access)
  - **AmazonS3FullAccess** (for S3 integration examples)
- Role name: **lambda-execution-role**
- Copy the Role ARN (needed for creating functions)

### Step 2: IAM User for Lambda Management
- Created IAM user with **programmatic access**
- Attached policies:
  - **AWSLambda_FullAccess** (manage Lambda functions)
  - **IAMReadOnlyAccess** (read roles for function creation)
- Generated access credentials (Access Key ID, Secret Access Key)
- Saved credentials securely in `.env` file

### Step 3: Create Simple Lambda Function (Manual Setup)
- Navigate to **Lambda** > **Create function**
- Function name: **hello-world-function**
- Runtime: **Python 3.12**
- Execution role: Use existing role **lambda-execution-role**
- Created function successfully
- This will be used for testing our boto3 operations

### Step 4: Environment File
Created `.env` file with the following variables:
```
AWS_ACCESS_KEY=your-access-key-id
AWS_SECRET_KEY=your-secret-access-key
AWS_REGION=us-east-2
LAMBDA_ROLE_ARN=arn:aws:iam::YOUR-ACCOUNT-ID:role/lambda-execution-role
```

### Step 5: Python Environment
- Installed boto3 library for AWS SDK
- Installed python-dotenv for secure credential loading

Now we're ready to explore Lambda operations programmatically.

---

## What You'll Learn

### Phase 1: Basic Lambda Operations (Read-Only)
- List all Lambda functions in your account
- Get function details (configuration, memory, timeout)
- Invoke functions and retrieve responses
- View function logs

### Phase 2: Function Management (Create & Deploy)
- Create Lambda functions programmatically
- Upload and update function code
- Delete functions
- Package Python code for deployment

### Phase 3: Configuration & Environment
- Set environment variables
- Configure memory and timeout settings
- Update execution role
- Manage function aliases and versions

### Phase 4: Event Sources & Triggers
- Create S3 event triggers
- Set up scheduled execution (EventBridge/CloudWatch)
- Configure API Gateway integration
- Process event data from different sources

### Phase 5: Advanced Features
- Create and attach Lambda layers
- View CloudWatch logs programmatically
- Implement error handling and retries
- Monitor function metrics
- Optimize performance and cost

## Lambda Architecture Overview

```
┌─────────────────────────────────────────────────────────┐
│  AWS LAMBDA ARCHITECTURE                                │
│                                                         │
│  Event Sources                Lambda Function           │
│  ┌──────────────┐            ┌──────────────┐          │
│  │ S3 Upload    │───────────>│              │          │
│  │ API Request  │───────────>│  Your Code   │          │
│  │ Schedule     │───────────>│  (Handler)   │          │
│  │ Manual Invoke│───────────>│              │          │
│  └──────────────┘            └──────┬───────┘          │
│                                     │                  │
│                                     ▼                  │
│  AWS Manages:              ┌──────────────┐            │
│  • Server provisioning     │   Outputs:   │            │
│  • Scaling                 │   • Return   │            │
│  • High availability       │   • S3 Write │            │
│  • Patching               │   • Database │            │
│  • Monitoring             └──────────────┘            │
│                                                         │
│  You Pay Only For:                                      │
│  • Number of requests (invocations)                     │
│  • Compute time (GB-seconds)                            │
└─────────────────────────────────────────────────────────┘
```

## Key Concepts

**Function**: Your code package with handler, runtime, and dependencies

**Handler**: Entry point function (e.g., `lambda_handler(event, context)`)

**Runtime**: Execution environment (Python 3.12, Node.js 20, Java 11, etc.)

**Event**: Input data passed to your function

**Context**: Runtime information (request ID, memory limit, etc.)

**Execution Role**: IAM role that grants permissions to your function

**Trigger**: Event source that invokes your function

**Cold Start**: First invocation after deployment (slower)

**Warm Start**: Subsequent invocations (faster, reuses container)

In [1]:
import boto3
from dotenv import load_dotenv
load_dotenv()
import os
import json

## 1. Environment Setup

Loading AWS credentials securely from environment variables using `.env` file.

**Security Best Practice:** Never hardcode credentials in your code!

In [2]:
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")
LAMBDA_ROLE_ARN = os.getenv("LAMBDA_ROLE_ARN")

## 2. Initialize Lambda Client

Create a boto3 Lambda client to interact with AWS Lambda service.

In [3]:
lambda_client = boto3.client('lambda',
                              region_name=AWS_REGION,
                              aws_access_key_id=ACCESS_KEY,
                              aws_secret_access_key=SECRET_KEY)

## PHASE 1: BASIC LAMBDA OPERATIONS

Starting with read-only operations to safely explore your Lambda functions.

### Function 1: List All Lambda Functions

Retrieve a list of all Lambda functions in your AWS account.

In [4]:
from botocore.exceptions import ClientError

def list_lambda_functions():
    """
    List all Lambda functions in your AWS account
    
    Returns:
        list: List of function names, or empty list if error
    """
    try:
        response = lambda_client.list_functions()
        
        functions = response.get('Functions', [])
        
        if not functions:
            print("No Lambda functions found in your account")
            return []
        
        print(f"Found {len(functions)} Lambda function(s):")
        
        function_names = []
        for func in functions:
            name = func['FunctionName']
            runtime = func['Runtime']
            memory = func['MemorySize']
            timeout = func['Timeout']
            last_modified = func['LastModified']
            
            print(f"Function: {name}")
            print(f"Runtime: {runtime}")
            print(f"Memory: {memory} MB")
            print(f"Timeout: {timeout} seconds")
            print(f"Last Modified: {last_modified}")
            
            function_names.append(name)
        
        return function_names
        
    except ClientError as e:
        print(f"ERROR: Failed to list functions - {e}")
        return []

# Test the function
function_names = list_lambda_functions()

Found 1 Lambda function(s):
Function: real_aws_lambda
Runtime: python3.12
Memory: 128 MB
Timeout: 3 seconds
Last Modified: 2026-01-29T07:50:16.061+0000


### Function 2: Get Function Configuration

Retrieve detailed configuration for a specific Lambda function including code size, environment variables, and execution role.

In [5]:
def get_function_config(function_name):
    """
    Get detailed configuration for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
    
    Returns:
        dict: Function configuration, or None if error
    """
    try:
        response = lambda_client.get_function_configuration(
            FunctionName=function_name
        )
        
        print(f"Configuration for '{function_name}':")
        
        # Basic info
        print(f"Function ARN: {response['FunctionArn']}")
        print(f"Runtime: {response['Runtime']}")
        print(f"Handler: {response['Handler']}")
        print(f"Memory Size: {response['MemorySize']} MB")
        print(f"Timeout: {response['Timeout']} seconds")
        print(f"Code Size: {response['CodeSize']:,} bytes")
        
        # Execution role
        print(f"Execution Role: {response['Role']}")
        
        # Environment variables
        env_vars = response.get('Environment', {}).get('Variables', {})
        if env_vars:
            print(f"Environment Variables:")
            for key, value in env_vars.items():
                print(f"  {key}: {value}")
        else:
            print(f"Environment Variables: None")
        
        # Layers
        layers = response.get('Layers', [])
        if layers:
            print(f"Layers:")
            for layer in layers:
                print(f"  - {layer['Arn']}")
        else:
            print(f"Layers: None")
        
        # VPC config
        vpc_config = response.get('VpcConfig', {})
        if vpc_config.get('VpcId'):
            print(f"VPC ID: {vpc_config['VpcId']}")
        else:
            print(f"VPC: Not configured")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to get function config - {e}")
        return None

# Test with the first function from our list
if function_names:
    config = get_function_config(function_names[0])

Configuration for 'real_aws_lambda':
Function ARN: arn:aws:lambda:us-east-2:445952351133:function:real_aws_lambda
Runtime: python3.12
Handler: lambda_function.lambda_handler
Memory Size: 128 MB
Timeout: 3 seconds
Code Size: 299 bytes
Execution Role: arn:aws:iam::445952351133:role/service-role/real_aws_lambda-role-k6ansg9t
Environment Variables: None
Layers: None
VPC: Not configured


### Function 3: Invoke Lambda Function

Execute a Lambda function and retrieve its response. This is the core operation for running serverless code.

In [6]:
def invoke_lambda(function_name, payload=None, invocation_type='RequestResponse'):
    """
    Invoke a Lambda function with optional payload
    
    Args:
        function_name (str): Name of the Lambda function
        payload (dict): Input data for the function (optional)
        invocation_type (str): 
            - 'RequestResponse' (default): Synchronous, wait for response
            - 'Event': Asynchronous, don't wait for response
            - 'DryRun': Validate parameters without executing
    
    Returns:
        dict: Function response including status code and payload
    """
    try:
        # Prepare payload
        if payload is None:
            payload = {}
        
        # Convert payload to JSON bytes
        payload_bytes = json.dumps(payload).encode('utf-8')
        
        print(f"Invoking '{function_name}' with invocation type: {invocation_type}")
        print(f"Payload: {json.dumps(payload, indent=2)}")
        print("-" * 80)
        
        # Invoke function
        response = lambda_client.invoke(
            FunctionName=function_name,
            InvocationType=invocation_type,
            Payload=payload_bytes
        )
        
        # Extract response details
        status_code = response['StatusCode']
        print(f"Status Code: {status_code}")
        
        # Read response payload
        if invocation_type == 'RequestResponse':
            response_payload = response['Payload'].read().decode('utf-8')
            response_data = json.loads(response_payload)
            
            print(f"Response Payload:")
            print(json.dumps(response_data, indent=2))
            
            # Check for function errors
            if 'FunctionError' in response:
                print(f"Function Error: {response['FunctionError']}")
            else:
                print(f"SUCCESS: Function executed successfully")
            
            return {
                'statusCode': status_code,
                'payload': response_data,
                'executedVersion': response.get('ExecutedVersion', '$LATEST')
            }
        else:
            print(f"Asynchronous invocation initiated")
            return {
                'statusCode': status_code,
                'message': 'Asynchronous invocation - no immediate response'
            }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to invoke function - {e}")
        return None

# Test with a simple payload
if function_names:
    test_payload = {
        "name": "Lambda Learner",
        "message": "Testing from boto3"
    }
    result = invoke_lambda(function_names[0], test_payload)

Invoking 'real_aws_lambda' with invocation type: RequestResponse
Payload: {
  "name": "Lambda Learner",
  "message": "Testing from boto3"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "\"Hello from Lambda!\""
}
SUCCESS: Function executed successfully


## PHASE 2: FUNCTION MANAGEMENT

Now we'll learn to create, update, and delete Lambda functions programmatically.

### Function 4: Create Lambda Function

Create a new Lambda function from Python code.

In [20]:
import zipfile
import io

def create_lambda_function(function_name, handler_code, handler='lambda_function.lambda_handler', 
                           runtime='python3.12', memory=128, timeout=30, description=''):
    """
    Create a new Lambda function with Python code
    
    Args:
        function_name (str): Name for the new function
        handler_code (str): Python code for the Lambda handler
        handler (str): Handler function path (default: lambda_function.lambda_handler)
        runtime (str): Python runtime version (default: python3.12)
        memory (int): Memory in MB (128-10240, default: 128)
        timeout (int): Timeout in seconds (1-900, default: 30)
        description (str): Function description
    
    Returns:
        dict: Created function configuration, or None if error
    
    Example handler_code:
        '''
        import json
        def lambda_handler(event, context):
            return {
                'statusCode': 200,
                'body': json.dumps('Hello from Lambda!')
            }
        '''
    """
    try:
        # Create in-memory ZIP file
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Add handler code to ZIP
            zip_file.writestr('lambda_function.py', handler_code)
        
        # Get ZIP bytes
        zip_bytes = zip_buffer.getvalue()
        
        print(f"Creating Lambda function '{function_name}'...")
        print(f"Runtime: {runtime}")
        print(f"Memory: {memory} MB")
        print(f"Timeout: {timeout} seconds")
        print(f"Code Size: {len(zip_bytes):,} bytes")
        
        # Create function
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime=runtime,
            Role=LAMBDA_ROLE_ARN,
            Handler=handler,
            Code={'ZipFile': zip_bytes},
            Description=description,
            Timeout=timeout,
            MemorySize=memory,
            Publish=True  # Publish initial version
        )
        
        print(f"SUCCESS: Function '{function_name}' created successfully")
        print(f"Function ARN: {response['FunctionArn']}")
        print(f"Version: {response['Version']}")
        print(f"State: {response['State']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceConflictException':
            print(f"ERROR: Function '{function_name}' already exists")
        elif error_code == 'InvalidParameterValueException':
            print(f"ERROR: Invalid parameter - {e.response['Error']['Message']}")
        else:
            print(f"ERROR: Failed to create function - {e}")
        return None

# Test: Create a simple "Hello World" Lambda function
hello_world_code = """
import json

def lambda_handler(event, context):
    \"\"\"
    Simple Hello World Lambda function
    \"\"\"
    # Get name from event or use default
    name = event.get('name', 'World')
    
    # Create response
    message = f"Hello, {name}! Created via boto3."
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': message,
            'requestId': context.aws_request_id
        })
    }
"""

# Create the function
new_function = create_lambda_function(
    function_name='boto3-hello-world',
    handler_code=hello_world_code,
    description='Hello World function created via boto3',
    memory=128,
    timeout=10
)

Creating Lambda function 'boto3-hello-world'...
Runtime: python3.12
Memory: 128 MB
Timeout: 10 seconds
Code Size: 387 bytes
SUCCESS: Function 'boto3-hello-world' created successfully
Function ARN: arn:aws:lambda:us-east-2:445952351133:function:boto3-hello-world
Version: 4
State: Pending


In [15]:
# Test the newly created function
print("Testing the new function:")
invoke_lambda('boto3-hello-world', {'name': 'Student'})

Testing the new function:
Invoking 'boto3-hello-world' with invocation type: RequestResponse
Payload: {
  "name": "Student"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "{\"message\": \"Hello, Student! Created via boto3.\", \"requestId\": \"2837f0ff-c199-4f85-8cde-aeea1da78457\"}"
}
SUCCESS: Function executed successfully


{'statusCode': 200,
 'payload': {'statusCode': 200,
  'body': '{"message": "Hello, Student! Created via boto3.", "requestId": "2837f0ff-c199-4f85-8cde-aeea1da78457"}'},
 'executedVersion': '$LATEST'}

### Function 5: Update Lambda Function Code

Update the code of an existing Lambda function.

In [16]:
def update_lambda_code(function_name, handler_code):
    """
    Update the code for an existing Lambda function
    
    Args:
        function_name (str): Name of the function to update
        handler_code (str): New Python code for the handler
    
    Returns:
        dict: Updated function configuration, or None if error
    """
    try:
        # Create in-memory ZIP file with new code
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.writestr('lambda_function.py', handler_code)
        
        zip_bytes = zip_buffer.getvalue()
        
        print(f"Updating code for '{function_name}'...")
        print(f"New code size: {len(zip_bytes):,} bytes")
        print("-" * 80)
        
        # Update function code
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_bytes,
            Publish=True  # Publish new version
        )
        
        print(f"SUCCESS: Function '{function_name}' code updated")
        print(f"New Version: {response['Version']}")
        print(f"Code Size: {response['CodeSize']:,} bytes")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to update function code - {e}")
        return None

# Test: Update the hello-world function with enhanced functionality
updated_code = """
import json
from datetime import datetime

def lambda_handler(event, context):
    \"\"\"
    Enhanced Hello World with timestamp
    \"\"\"
    name = event.get('name', 'World')
    timestamp = datetime.utcnow().isoformat()
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f"Hello, {name}! (Updated version)",
            'timestamp': timestamp,
            'requestId': context.aws_request_id,
            'version': 'v2.0'
        })
    }
"""

# Update the function
updated = update_lambda_code('boto3-hello-world', updated_code)

Updating code for 'boto3-hello-world'...
New code size: 401 bytes
--------------------------------------------------------------------------------
SUCCESS: Function 'boto3-hello-world' code updated
New Version: 3
Code Size: 401 bytes
Last Modified: 2026-01-29T08:04:44.000+0000


In [17]:
# Test the updated function
if updated:
    print("Testing updated function:")
    invoke_lambda('boto3-hello-world', {'name': 'Updated Test'})

Testing updated function:
Invoking 'boto3-hello-world' with invocation type: RequestResponse
Payload: {
  "name": "Updated Test"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "{\"message\": \"Hello, Updated Test! (Updated version)\", \"timestamp\": \"2026-01-29T08:04:53.770931\", \"requestId\": \"f14e3bf3-b67d-49f3-80a1-a92d973db00b\", \"version\": \"v2.0\"}"
}
SUCCESS: Function executed successfully


### Function 6: Delete Lambda Function

Delete a Lambda function. Use with caution - this operation is permanent.

In [18]:
def delete_lambda_function(function_name):
    """
    Delete a Lambda function
    
    Args:
        function_name (str): Name of the function to delete
    
    Returns:
        bool: True if deleted successfully, False otherwise
    
    WARNING: This operation is permanent and cannot be undone!
    """
    try:
        print(f"WARNING: Deleting function '{function_name}'...")
        print("This operation is PERMANENT and cannot be undone!")
        
        # Delete function
        lambda_client.delete_function(FunctionName=function_name)
        
        print(f"SUCCESS: Function '{function_name}' deleted successfully")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found (already deleted?)")
        else:
            print(f"ERROR: Failed to delete function - {e}")
        return False

# Example: Delete the test function we created
# UNCOMMENT TO TEST (be careful - this is permanent):
delete_lambda_function('boto3-hello-world')

# Verify deletion by listing functions
list_lambda_functions()

This operation is PERMANENT and cannot be undone!
SUCCESS: Function 'boto3-hello-world' deleted successfully
Found 1 Lambda function(s):
Function: real_aws_lambda
Runtime: python3.12
Memory: 128 MB
Timeout: 3 seconds
Last Modified: 2026-01-29T07:50:16.061+0000


['real_aws_lambda']

## PHASE 3: CONFIGURATION & ENVIRONMENT

Now we'll learn to configure Lambda functions by setting environment variables, adjusting memory and timeout, and managing function aliases and versions.

### Function 7: Set Environment Variables

Configure environment variables for a Lambda function. These are accessible within your function code via `os.environ`.

In [21]:
def set_environment_variables(function_name, env_vars):
    """
    Set or update environment variables for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        env_vars (dict): Dictionary of environment variables (key-value pairs)
    
    Returns:
        dict: Updated function configuration, or None if error
    
    Example:
        set_environment_variables('my-function', {
            'DATABASE_URL': 'postgresql://...',
            'API_KEY': 'secret-key',
            'DEBUG': 'true'
        })
    """
    try:
        print(f"Setting environment variables for '{function_name}'...")
        print(f"Variables to set:")
        for key, value in env_vars.items():
            # Mask sensitive values in output
            display_value = value if len(value) < 20 else f"{value[:10]}...{value[-5:]}"
            print(f"  {key}: {display_value}")
        
        # Update function configuration
        response = lambda_client.update_function_configuration(
            FunctionName=function_name,
            Environment={
                'Variables': env_vars
            }
        )
        
        print(f"SUCCESS: Environment variables updated")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to set environment variables - {e}")
        return None

# Test: Add environment variables to our test function
test_env = {
    'ENVIRONMENT': 'development',
    'LOG_LEVEL': 'INFO',
    'FEATURE_FLAG': 'enabled'
}

env_result = set_environment_variables('boto3-hello-world', test_env)

Setting environment variables for 'boto3-hello-world'...
Variables to set:
  ENVIRONMENT: development
  LOG_LEVEL: INFO
  FEATURE_FLAG: enabled
SUCCESS: Environment variables updated
Last Modified: 2026-01-29T08:08:46.000+0000


### Function 8: Update Function Configuration

Update memory allocation and timeout settings for a Lambda function to optimize performance and cost.

In [22]:
def update_function_configuration(function_name, memory=None, timeout=None, description=None):
    """
    Update configuration settings for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        memory (int): Memory in MB (128-10240, multiples of 64)
        timeout (int): Timeout in seconds (1-900)
        description (str): Function description
    
    Returns:
        dict: Updated function configuration, or None if error
    
    Note: 
        - More memory = more CPU power (proportional)
        - Memory affects cost: $0.0000166667 per GB-second
        - Timeout max is 15 minutes (900 seconds)
    """
    try:
        # Build update parameters
        update_params = {'FunctionName': function_name}
        
        if memory is not None:
            if memory < 128 or memory > 10240:
                print(f"ERROR: Memory must be between 128 and 10240 MB")
                return None
            update_params['MemorySize'] = memory
        
        if timeout is not None:
            if timeout < 1 or timeout > 900:
                print(f"ERROR: Timeout must be between 1 and 900 seconds")
                return None
            update_params['Timeout'] = timeout
        
        if description is not None:
            update_params['Description'] = description
        
        if len(update_params) == 1:
            print("ERROR: No configuration changes specified")
            return None
        
        print(f"Updating configuration for '{function_name}'...")
        if memory:
            print(f"Memory: {memory} MB")
        if timeout:
            print(f"Timeout: {timeout} seconds")
        if description:
            print(f"Description: {description}")
        
        # Update function configuration
        response = lambda_client.update_function_configuration(**update_params)
        
        print(f"SUCCESS: Configuration updated")
        print(f"Current Memory: {response['MemorySize']} MB")
        print(f"Current Timeout: {response['Timeout']} seconds")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to update configuration - {e}")
        return None

# Test: Update memory and timeout for better performance
config_result = update_function_configuration(
    'boto3-hello-world',
    memory=256,  # Increase from 128 to 256 MB
    timeout=30,  # Increase from 10 to 30 seconds
    description='Hello World function - updated configuration'
)

Updating configuration for 'boto3-hello-world'...
Memory: 256 MB
Timeout: 30 seconds
Description: Hello World function - updated configuration
SUCCESS: Configuration updated
Current Memory: 256 MB
Current Timeout: 30 seconds
Last Modified: 2026-01-29T08:09:06.000+0000


### Function 9: Create Function Alias

Create an alias for a Lambda function version. Aliases are pointers to specific versions and can be used for traffic splitting and blue/green deployments.

In [25]:
def create_function_alias(function_name, alias_name, version, description=''):
    """
    Create an alias pointing to a specific function version
    
    Args:
        function_name (str): Name of the Lambda function
        alias_name (str): Name for the alias (e.g., 'prod', 'staging', 'dev')
        version (str): Version number to point to (or '$LATEST')
        description (str): Alias description
    
    Returns:
        dict: Created alias configuration, or None if error
    
    Use cases:
        - Blue/Green deployments: Switch traffic between versions
        - Environment separation: prod, staging, dev aliases
        - Traffic splitting: Route percentage to different versions
    """
    try:
        print(f"Creating alias '{alias_name}' for '{function_name}'...")
        print(f"Points to version: {version}")
        
        # Create alias
        response = lambda_client.create_alias(
            FunctionName=function_name,
            Name=alias_name,
            FunctionVersion=version,
            Description=description
        )
        
        print(f"SUCCESS: Alias '{alias_name}' created")
        print(f"Alias ARN: {response['AliasArn']}")
        print(f"Function Version: {response['FunctionVersion']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceConflictException':
            print(f"ERROR: Alias '{alias_name}' already exists")
        elif error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' or version '{version}' not found")
        else:
            print(f"ERROR: Failed to create alias - {e}")
        return None

# Test: Create a 'prod' alias pointing to version 4
alias_result = create_function_alias(
    'boto3-hello-world',
    alias_name='prod',
    version='4',
    description='Production version'
)

Creating alias 'prod' for 'boto3-hello-world'...
Points to version: 4
SUCCESS: Alias 'prod' created
Alias ARN: arn:aws:lambda:us-east-2:445952351133:function:boto3-hello-world:prod
Function Version: 4


### Function 10: List Function Versions

List all versions of a Lambda function to track deployment history.

In [26]:
def list_function_versions(function_name):
    """
    List all versions of a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
    
    Returns:
        list: List of version configurations, or empty list if error
    
    Note: 
        - $LATEST is always available (unpublished changes)
        - Published versions are immutable
        - Version numbers are sequential (1, 2, 3, ...)
    """
    try:
        print(f"Listing versions for '{function_name}'...")
        print("-" * 80)
        
        # List all versions
        response = lambda_client.list_versions_by_function(
            FunctionName=function_name
        )
        
        versions = response.get('Versions', [])
        
        if not versions:
            print(f"No versions found for '{function_name}'")
            return []
        
        print(f"Found {len(versions)} version(s):")
        
        version_list = []
        for version in versions:
            version_num = version['Version']
            code_size = version['CodeSize']
            last_modified = version['LastModified']
            description = version.get('Description', 'No description')
            
            print(f"Version: {version_num}")
            print(f"Code Size: {code_size:,} bytes")
            print(f"Last Modified: {last_modified}")
            print(f"Description: {description}")
            
            version_list.append({
                'version': version_num,
                'codeSize': code_size,
                'lastModified': last_modified,
                'description': description
            })
        
        return version_list
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to list versions - {e}")
        return []

# Test: List all versions of our test function
versions = list_function_versions('boto3-hello-world')

Listing versions for 'boto3-hello-world'...
--------------------------------------------------------------------------------
Found 2 version(s):
Version: $LATEST
Code Size: 387 bytes
Last Modified: 2026-01-29T08:09:06.000+0000
Description: Hello World function - updated configuration
Version: 4
Code Size: 387 bytes
Last Modified: 2026-01-29T08:08:37.443+0000
Description: Hello World function created via boto3
